In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('glassdoor_jobs.csv')

In [29]:
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


#salary:
    1. Min & Max separate columns
    2. Remove glassdoor est
#Company Name
    1. Keep name only, remove rating
#Location
    1. City and State separate
#Founded
    1. Company Age
#Job Description
    1. Parse JD for keywords

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         956 non-null    int64  
 1   Job Title          956 non-null    object 
 2   Salary Estimate    956 non-null    object 
 3   Job Description    956 non-null    object 
 4   Rating             956 non-null    float64
 5   Company Name       956 non-null    object 
 6   Location           956 non-null    object 
 7   Headquarters       956 non-null    object 
 8   Size               956 non-null    object 
 9   Founded            956 non-null    int64  
 10  Type of ownership  956 non-null    object 
 11  Industry           956 non-null    object 
 12  Sector             956 non-null    object 
 13  Revenue            956 non-null    object 
 14  Competitors        956 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 112.2+ KB


In [31]:
df.isna().sum()

Unnamed: 0           0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
dtype: int64

In [32]:
(df['Salary Estimate'] == '-1').sum()

214

In [33]:
df = df[df['Salary Estimate']!='-1']

# Regex

### Cleaning Salary

In [34]:
regex_df = df.copy()

In [35]:
#Glassdoor estimate removal
salary = regex_df['Salary Estimate'].apply(lambda x:x.split('(')[0])

In [36]:
regex_df['per_hour'] = salary.apply(lambda x: 1 if "per hour" in x.lower() else 0)

In [37]:
regex_df['employer_provided'] = salary.apply(lambda x: 1 if "employer provided salary:" in x.lower() else 0)

In [38]:
#'k' and '$' removal
remove_kd = salary.apply(lambda x:x.replace('K','').replace('$', ''))

In [39]:
remove_text = remove_kd.apply(lambda x: x.lower().replace('per hour', '').replace('employer provided salary:', ''))

In [40]:
regex_df['min salary'] = remove_text.apply(lambda x: x.split('-')[0])

In [41]:
regex_df['max_salary'] =  remove_text.apply(lambda x:x.split('-')[1])

### Company Name Parse

In [42]:
remove_n = regex_df['Company Name'].apply(lambda x: x.replace('\n', ','))

In [43]:
regex_df['company_txt'] = remove_n.apply(lambda x: x.split(',')[0])

# Location

In [44]:
extract_state = regex_df['Location'].apply(lambda x: x.split(',')[1])
extract_state = extract_state.apply(lambda x: x.replace('Los Angeles', 'CA'))

In [45]:
extract_city = regex_df['Location'].apply(lambda x: x.split(',')[0])

In [46]:
regex_df['State'] = extract_state
regex_df['City'] = extract_city

In [47]:
# It was throwing an error with x.split(',')[1], so used [-1] to get the states values.
extract_hq_state = regex_df['Headquarters'].apply(lambda x: x.split(',')[-1])

In [48]:
#Test since above code wasn't working, it worked with [-1]
#for x in regex_df['Headquarters']:
    #split_result = x.split(',')[1]
    #print((split_result))

In [49]:
temp_df = pd.DataFrame({'extract_state' : extract_state, 'extract_hq_state' : extract_hq_state})


In [50]:
same_state_hq = temp_df.apply(lambda x: 1 if x.extract_state == x.extract_hq_state else 0, axis =1)


In [51]:
regex_df['same_state_hq'] = same_state_hq

# Founded

In [52]:
from datetime import datetime

In [53]:
year = datetime.now().year

In [54]:
company_age = []
for x in regex_df.Founded:
    if x == -1:
        company_age.append('NAN')
    else:
        age = year - x
        company_age.append(age)


In [55]:
regex_df['company_age'] = company_age

In [56]:
regex_df

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,per_hour,employer_provided,min salary,max_salary,company_txt,State,City,same_state_hq,company_age
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,Tecolote Research,NM,Albuquerque,0,50
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,University of Maryland Medical System,MD,Linthicum,1,39
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80,90,KnowBe4,FL,Clearwater,1,13
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,PNNL,WA,Richland,1,58
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,Affinity Solutions,NY,New York,1,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),"Pfizer, AstraZeneca, Merck",0,0,58,111,GSK,MA,Cambridge,0,193
951,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini",0,0,72,133,Eventbrite,TN,Nashville,0,17
952,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,-1,0,0,56,91,Software Engineering Institute,PA,Pittsburgh,1,39
953,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1,0,0,95,160,Numeric,PA,Allentown,1,NAN


# JD Parse (Added each of these series to regex_df separately)

In [59]:
regex_df['jd_python'] = regex_df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() or 'py' in x.lower() else 0)

In [63]:
regex_df

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,per_hour,employer_provided,min salary,max_salary,company_txt,State,City,same_state_hq,company_age,jd_python,jd_sql
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,Tecolote Research,NM,Albuquerque,0,50,1,0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,University of Maryland Medical System,MD,Linthicum,1,39,1,0
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80,90,KnowBe4,FL,Clearwater,1,13,1,1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,PNNL,WA,Richland,1,58,1,0
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,Affinity Solutions,NY,New York,1,25,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),"Pfizer, AstraZeneca, Merck",0,0,58,111,GSK,MA,Cambridge,0,193,0,0
951,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini",0,0,72,133,Eventbrite,TN,Nashville,0,17,1,1
952,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,-1,0,0,56,91,Software Engineering Institute,PA,Pittsburgh,1,39,0,0
953,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1,0,0,95,160,Numeric,PA,Allentown,1,NAN,0,0


In [65]:
regex_df.jd_python.value_counts()

1    418
0    324
Name: jd_python, dtype: int64

In [66]:
regex_df['jd_sql'] = regex_df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() or 'mysql' in x.lower() else 0)

In [67]:
regex_df.jd_sql.value_counts()

1    380
0    362
Name: jd_sql, dtype: int64

In [68]:
regex_df['jd_excel'] = regex_df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() or 'msexcel' in x.lower() else 0)

In [69]:
regex_df.jd_excel.value_counts()

1    388
0    354
Name: jd_excel, dtype: int64

In [70]:
regex_df['jd_spark'] = regex_df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [71]:
regex_df.jd_spark.value_counts()

0    575
1    167
Name: jd_spark, dtype: int64

In [72]:
regex_df['jd_apache'] = regex_df['Job Description'].apply(lambda x: 1 if 'apache' in x.lower() else 0)

In [73]:
regex_df.jd_apache.value_counts()

0    709
1     33
Name: jd_apache, dtype: int64

In [74]:
regex_df['jd_r'] = regex_df['Job Description'].apply(lambda x: 1 if 'r' in x.lower() or ' r studio' in x.lower() or 'r-studio' in x.lower()  else 0)

In [75]:
regex_df.jd_r.value_counts()

1    742
Name: jd_r, dtype: int64

In [76]:
regex_df['jd_tensor'] = regex_df['Job Description'].apply(lambda x: 1 if 'tensorflow' in x.lower() or 'tensor' in x.lower() or 'tensor flow' in x.lower() else 0)

In [77]:
regex_df.jd_tensor.value_counts()

0    670
1     72
Name: jd_tensor, dtype: int64

In [78]:
regex_df['jd_jupyter'] = regex_df['Job Description'].apply(lambda x: 1 if 'jupyter' in x.lower() or 'notebook' in x.lower() else 0)

In [79]:
regex_df.jd_jupyter.value_counts()

0    697
1     45
Name: jd_jupyter, dtype: int64

In [80]:
regex_df['jd_git'] = regex_df['Job Description'].apply(lambda x: 1 if 'git' in x.lower() or 'github' in x.lower() else 0)

In [81]:
regex_df.jd_git.value_counts()

0    581
1    161
Name: jd_git, dtype: int64

In [82]:
regex_df['jd_library'] = regex_df['Job Description'].apply(lambda x: 1 if 'pandas' in x.lower() or 'numpy' in x.lower() or 'scikit' in x.lower() or 'scikit-learn' in x.lower() else 0)

In [83]:
regex_df.jd_library.value_counts()

0    672
1     70
Name: jd_library, dtype: int64

In [84]:
regex_df['jd_aws'] = regex_df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [85]:
regex_df.jd_aws.value_counts()

0    566
1    176
Name: jd_aws, dtype: int64

In [86]:
regex_df['jd_cloud'] = regex_df['Job Description'].apply(lambda x: 1 if 'cloud' in x.lower() or 'google cloud' in x.lower() else 0)

In [87]:
regex_df.jd_cloud.value_counts()

0    576
1    166
Name: jd_cloud, dtype: int64

In [88]:
regex_df['jd_linux'] = regex_df['Job Description'].apply(lambda x: 1 if 'linux' in x.lower() else 0)

In [89]:
regex_df.jd_linux.value_counts()

0    686
1     56
Name: jd_linux, dtype: int64

In [90]:
regex_df = regex_df.drop(['Unnamed: 0'], axis = 1)

# Job Title Parse

In [93]:
import regex as re

In [94]:
clean_title = []
for x in regex_df['Job Title']:
    match = re.findall(r'\bdata scientist\b', x, flags = re.IGNORECASE)
    match_2 = re.findall(r'\bdata science\b', x, flags = re.IGNORECASE)
    match_3 = re.findall(r'\bdata analyst\b', x, flags = re.IGNORECASE)
    match_4 = re.findall(r'\bdata analytics\b', x, flags = re.IGNORECASE)
    match_5 = re.findall(r'\bdata engineer\b', x, flags = re.IGNORECASE)
    match_6 = re.findall(r'\bresearch scientist\b', x, flags = re.IGNORECASE)
    match_7 = re.findall(r'\bmachine learning\b', x, flags = re.IGNORECASE)
    match_8 = re.findall(r'\bdirector\b', x, flags = re.IGNORECASE)
    match_9 = re.findall(r'\bmanager\b', x, flags = re.IGNORECASE)
    match_10 = re.findall(r'\bconsultant\b', x, flags = re.IGNORECASE)
    match_11 = re.findall(r'\bassociate director\b', x, flags = re.IGNORECASE)
    match_12 = re.findall(r'\banalyst\b', x, flags = re.IGNORECASE)
    match_13 = re.findall(r'\bdata engineering\b', x, flags = re.IGNORECASE)
    if match or match_2:
        clean_title.append(x.replace(x, 'Data Scientist'))
        #regex_df[job_title] = job_title.append(x.replace(x, 'Data Scientist'))
    elif match_3:
        clean_title.append(x.replace(x, 'Data Analyst'))
    elif match_4:
        clean_title.append(x.replace(x, 'Data Analyst'))
    elif match_5:
        clean_title.append(x.replace(x, 'Data Engineer'))
    elif match_6:
        clean_title.append(x.replace(x, 'Research Scientist'))
    elif match_7:
        clean_title.append(x.replace(x, 'Machine Learning Engineer'))
    elif match_8:
        clean_title.append(x.replace(x, 'Director'))
    elif match_9:
        clean_title.append(x.replace(x, 'Manager'))
    elif match_10:
        clean_title.append(x.replace(x, 'Consultant'))
    elif match_11:
        clean_title.append(x.replace(x, 'Associate Director'))
    elif match_12:
        clean_title.append(x.replace(x, 'Data Analyst'))
    elif match_13:
        clean_title.append(x.replace(x, 'Data Engineer'))
    else:
        clean_title.append(x.replace(x, 'NA'))

In [95]:
regex_df['clean_title'] = clean_title

In [96]:
regex_df.clean_title.value_counts()

Data Scientist               313
NA                           143
Data Engineer                115
Data Analyst                 106
Research Scientist            22
Machine Learning Engineer     15
Manager                       13
Director                       8
Consultant                     7
Name: clean_title, dtype: int64

In [97]:
regex_df.clean_title.value_counts().sum()

742

In [98]:
regex_df.clean_title.value_counts().to_csv('Clean_titles.csv', index = False)

# JD Length

In [99]:
regex_df['jd_length'] = regex_df['Job Description'].apply(lambda x: len(x))

# Competitor Count

In [100]:
regex_df['comp_count'] = regex_df['Competitors'].apply(lambda x: len(x.split(',')) if x != '-1' else 0)

# Per hour to annual


In [101]:
regex_df['min salary'] = pd.to_numeric(regex_df['min salary'], errors='coerce')
regex_df['max_salary'] = pd.to_numeric(regex_df['max_salary'], errors='coerce')

In [102]:
regex_df['min salary'] = regex_df.apply(lambda x: round(x['min salary']*1.92,0) if x.per_hour == 1 else x['min salary'], axis =1)

In [103]:
regex_df['max_salary'] = regex_df.apply(lambda x: round(x['max_salary']*1.92,0) if x.per_hour == 1 else x['max_salary'], axis =1)

In [104]:
regex_df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,per_hour,employer_provided,min salary,max_salary,company_txt,State,City,same_state_hq,company_age,jd_python,jd_sql,jd_excel,jd_spark,jd_apache,jd_r,jd_tensor,jd_jupyter,jd_git,jd_library,jd_aws,jd_cloud,jd_linux,clean_title,jd_length,comp_count
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53.0,91.0,Tecolote Research,NM,Albuquerque,0,50,1,0,1,0,0,1,0,0,0,0,0,0,0,Data Scientist,2536,0
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63.0,112.0,University of Maryland Medical System,MD,Linthicum,1,39,1,0,0,0,0,1,0,0,0,0,0,0,0,Data Scientist,4783,0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80.0,90.0,KnowBe4,FL,Clearwater,1,13,1,1,1,1,0,1,0,0,0,0,0,0,0,Data Scientist,3461,0
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56.0,97.0,PNNL,WA,Richland,1,58,1,0,0,0,0,1,0,0,0,0,0,0,0,Data Scientist,3883,3
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86.0,143.0,Affinity Solutions,NY,New York,1,25,1,1,1,0,0,1,0,0,0,0,0,1,0,Data Scientist,2728,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),"Pfizer, AstraZeneca, Merck",0,0,58.0,111.0,GSK,MA,Cambridge,0,193,0,0,0,0,0,1,0,0,0,0,1,0,0,NA,6162,3
951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini",0,0,72.0,133.0,Eventbrite,TN,Nashville,0,17,1,1,0,1,0,1,0,0,0,0,1,1,0,Data Engineer,6130,3
952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,-1,0,0,56.0,91.0,Software Engineering Institute,PA,Pittsburgh,1,39,0,0,1,0,0,1,0,0,0,0,0,0,0,NA,3078,0
953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1,0,0,95.0,160.0,Numeric,PA,Allentown,1,NAN,0,0,1,0,0,1,0,0,1